## CIFAR10 с MobileNet V2

In [1]:
import copy

from typing import Dict, Optional

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch

from sklearn.metrics import confusion_matrix
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms as T
from tqdm.notebook import tqdm

### Загрузка данных
#### Предобработка данных заключается в первоначальном изменении исходного изображения 32 х 32 до 224 х 224, так как это минимальные небходимый размер для сети MobileNetV2 из хаба pytorch, преобразовании их к тензору pytorch-а и приведению мат ожидания и дисперсии каждого канала изображения к следующим значенениям
#### Для обучающего датасета также будут использоваться аугментации:
1. ColorJitter
2. RandomEqualize
3. RandomHorizontalFlip и RandomVerticalFlip


In [2]:
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

transforms = {
    'train': T.Compose([
        T.Resize(224),
        T.ColorJitter(brightness=.5, hue=.3),
        T.RandomEqualize(),
        T.RandomHorizontalFlip(),
        T.RandomVerticalFlip(),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD),
    ]),
    'valid': T.Compose([
        T.Resize(224),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD),
    ]),
}

batch_sizes = {'train': 288, 'valid': 480}

datasets = {
    'train': datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms['train']),
    'valid': datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms['valid']),
}

loaders = {
    'train': DataLoader(datasets['train'], batch_size=batch_sizes['train'], shuffle=True, num_workers=2),
    'valid': DataLoader(datasets['valid'], batch_size=batch_sizes['valid'], shuffle=False, num_workers=2),
}

Files already downloaded and verified
Files already downloaded and verified


### Функция обучения нейросети
В функции осуществляется обучение и валидация.

Входные параметры:
1. Количество эпох;
2. Модель;
3. Оптимизатор;
4. Функция потерь;
5. Лоадер;
6. Девайс;
7. Скэжулер;
8. Число шагов с аккамуляцией.

In [4]:
def train(
  num_epochs: int,
  model: nn.Module,
  optimizer: optim.Optimizer,
  criterion: nn.Module,
  loader: DataLoader,
  device: torch.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
  scheduler = None,
  accumulation: Optional[int] = 1,
):
  model = model.to(device)
  min_eval_loss = float('inf')
  for epoch in range(num_epochs):
    train_loss = train_epoch(
      model=model, optimizer=optimizer, criterion=criterion, loader=loader['train'],
      device=device, scheduler=scheduler, accumulation=accumulation
    )
    eval_loss = valid_epoch(
        model=model, criterion=criterion, loader=loader['valid'], device=device
    )
    if eval_loss < min_eval_loss:
      best_model_wts = copy.deepcopy(model.state_dict())

    print(f'Epoch: {epoch + 1:>2}/{num_epochs}\tTrain loss: {train_loss:<10.4f}\tEval loss: {eval_loss:<10.4f}')

  print(f'Best val Acc: {min_eval_loss:4f}')

  # load best model weights
  model.load_state_dict(best_model_wts)
  return model

def train_epoch(
  model: nn.Module,
  optimizer: optim.Optimizer,
  criterion: nn.Module,
  loader: Dict[str, DataLoader],
  device: torch.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
  scheduler = None,
  accumulation: Optional[int] = 1,
):
  model = model.train().to(device)
  total_loss = 0
  len_dataloader = len(loader)

  for i, (imgs, labels) in tqdm(enumerate(loader, 1), leave=False, total=len_dataloader):
    imgs, labels = imgs.to(device), labels.to(device)

    logits = model(imgs)

    loss = criterion(logits, labels)
    loss.backward()
    
    if not i % accumulation or i == len_dataloader:
      optimizer.step()
      optimizer.zero_grad()
      if scheduler is not None:
        scheduler.step()
    
    total_loss += loss.item()

  return total_loss / len_dataloader

@torch.no_grad()
def valid_epoch(
  model: nn.Module,
  criterion: nn.Module,
  loader: DataLoader,
  device: torch.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
):
  model = model.eval().to(device)
  total_loss = 0
  len_dataloader = len(loader)

  for imgs, labels in tqdm(loader, leave=False, total=len_dataloader):
    imgs, labels = imgs.to(device), labels.to(device)

    logits = model(imgs)

    loss = criterion(logits, labels)
    
    total_loss += loss.item()

  return total_loss / len_dataloader

### Модель

Для обучения была взята MobileNetV2.

Изменена последняя часть сети - классификатор. Также были заморожены первые слои метки.

In [5]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)

model.classifier = nn.Sequential(
  nn.Dropout(p=0.1),
  nn.Linear(in_features=1280, out_features=128),
  nn.LeakyReLU(0.05),
  nn.Dropout(p=0.1),
  nn.Linear(in_features=128, out_features=10),
)

freeze_layer = 15
for x in list(model.features.parameters())[:freeze_layer]:
  x.requires_grad = False
for x in list(model.features[freeze_layer:].parameters())[freeze_layer:]:
  x.requires_grad = True
for x in model.classifier.parameters():
  x.requires_grad = True

### Параметры обучения

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = optim.AdamW(list(model.parameters())[freeze_layer:], lr=3e-4)
criterion = nn.CrossEntropyLoss()
accumulation = 2
num_epochs = 30

In [ ]:
best_model = train(num_epochs, model, optimizer, criterion, loaders, device, accumulation=accumulation)

### Построение матрицы классификации на валидационном датасете

In [9]:
def compute_confusion_matrix(model, loader):
  conf_matrix = np.zeros((10, 10))
  for inputs, labels in tqdm(loader, leave=False):
    inputs, labels = inputs.to(device), labels.to(device)
    logits = model(inputs)

    preds = torch.argmax(logits, dim=1)
    conf_matrix += confusion_matrix(
      labels.cpu().numpy(), preds.cpu().numpy()
    )
  return conf_matrix

In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
best_model.eval()

In [10]:
conf_m = pd.DataFrame(compute_confusion_matrix(best_model, loaders['train']), index=classes, columns=classes)
plt.figure(figsize = (10,10))
sns.heatmap(conf_m, annot=True)

In [ ]:
conf_m = pd.DataFrame(compute_confusion_matrix(best_model, loaders['valid']), index=classes, columns=classes)
plt.figure(figsize = (10,10))
sns.heatmap(conf_m, annot=True)